In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [2]:
print("Num GPUs Available: ",
      len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [3]:
TRAIN_DIR = './dataset/sdd/train/'
TEST_DIR = './dataset/sdd/test/'
IMG_SIZE = (150, 150)
BATCH_SIZE = 128
SEED = 1337

In [4]:
# Training generator
train_datagen = ImageDataGenerator(rescale=1. / 255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

train_ds = train_datagen.flow_from_directory(TRAIN_DIR,
                                             target_size=IMG_SIZE,
                                             batch_size=BATCH_SIZE,
                                             seed=SEED,
                                             shuffle=True,
                                             class_mode='categorical')

# Test generator
test_datagen = ImageDataGenerator(rescale=1. / 255)

test_ds = test_datagen.flow_from_directory(TEST_DIR,
                                           target_size=IMG_SIZE,
                                           batch_size=BATCH_SIZE,
                                           seed=SEED,
                                           shuffle=False,
                                           class_mode='categorical')

train_num = train_ds.samples
test_num = test_ds.samples

Found 16229 images belonging to 120 classes.
Found 4111 images belonging to 120 classes.


In [5]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(150, 150, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))

model.add(Conv2D(256, (3, 3), padding='same'))
model.add(Activation('relu'))

model.add(GlobalAveragePooling2D())
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Activation('relu'))
model.add(Dense(120))
model.add(Activation('softmax'))

In [6]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['acc'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 32)      896       
_________________________________________________________________
activation (Activation)      (None, 148, 148, 32)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 32)        9248      
_________________________________________________________________
activation_1 (Activation)    (None, 72, 72, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 36, 36, 64)        1

In [7]:
EPOCHS = 1

checkpoint_path = "checkpoint/dog/cp.ckpt"

model.load_weights(checkpoint_path)

cp_callback = ModelCheckpoint(filepath=checkpoint_path,
                              save_weights_only=True,
                              save_best_only=True,
                              mode='min',
                              verbose=1)

history = model.fit_generator(train_ds,
                              steps_per_epoch=train_num // BATCH_SIZE,
                              epochs=EPOCHS,
                              validation_data=test_ds,
                              validation_steps=test_num // BATCH_SIZE,
                              callbacks=[cp_callback])

In [8]:
checkpoint_path = "checkpoint/dog/cp.ckpt"

model.load_weights(checkpoint_path)

model.evaluate(test_ds, batch_size=BATCH_SIZE)

model.save('boku_net.h5')

33/33 [==============================] - 6s 179ms/step - loss: 2.5759 - acc: 0.3972


In [9]:
model = tf.keras.models.load_model('boku_net.h5')

model.evaluate(test_ds, batch_size=BATCH_SIZE)

33/33 [==============================] - 6s 179ms/step - loss: 2.5759 - acc: 0.3972


[2.5759358406066895, 0.3972269594669342]